## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose start date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_started_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/started_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'ALL_spots_wo_CB_cancellation_requested'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_and_didnt_last_month_vars + did_something_last_month_vars


#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
for model_number in model_numbers[1:]:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_last_month_vars, did_something_before_and_didnt_last_month_vars) = \
    get_started_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_and_didnt_last_month_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)

# Model 1: posts_on_facebook

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 31703 periods, 2527 subjects, 211 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2527
 number of periods = 31703
  number of events = 211
partial log-likelihood = -1326.65
  time fit was run = 2021-12-21 10:06:37 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Monthly                                              0.95       2.58       0.24             0.48             1.42                 1.61                 4.14
New.email.subscriber.signups.last.month.div10       -0.12       0.89       0.05            -0.22            -0.01                 0.80                 0.99
SpotHopper_site_not_live_yet                         0.34       1.40       0.25            -0.16             0.84                 0.85                 2.31
Website.Views.last.month.total.log2                 -0.06       0.94       0.05            -0.15             0.03                 0.86                 1.03
had_added_events_manually_edited_events_before_...   0.02       1.02       0.18            -0.34             0.38                 0.71                 1.46
had_added_events_manually_edited_events_last_3_...  -0.63       0.53       0.29            -1.20            -0.07                 0.30                 0.94
had_catering_submissions_before_and_didnt_last_...   0.11       1.11       0.26            -0.40             0.62                 0.67                 1.85
had_catering_submissions_last_3_months              -0.56       0.57       0.23            -1.02            -0.11                 0.36                 0.90
had_clicked_emails_before_and_didnt_last_3_months    0.70       2.00       0.18             0.34             1.05                 1.41                 2.86
had_clicked_emails_last_3_months                     0.07       1.08       0.20            -0.31             0.46                 0.73                 1.58
had_emails_sent_manually_scheduled_emails_sent_...   0.09       1.09       0.17            -0.24             0.42                 0.78                 1.52
had_emails_sent_manually_scheduled_emails_sent_...  -0.32       0.72       0.18            -0.68             0.04                 0.51                 1.04
had_online_orders_before_and_didnt_last_month        0.57       1.76       0.31            -0.05             1.18                 0.95                 3.27
had_online_orders_last_month                        -0.20       0.82       0.33            -0.85             0.45                 0.43                 1.57
had_other_non_contactless_menu_qr_flyer_scans_b...   0.39       1.47       0.37            -0.34             1.11                 0.71                 3.03
had_other_non_contactless_menu_qr_flyer_scans_l...  -0.71       0.49       0.55            -1.78             0.37                 0.17                 1.44
had_posts_on_facebook_before_and_didnt_last_month    0.41       1.50       0.17             0.07             0.74                 1.08                 2.09
had_posts_on_facebook_last_month                    -0.14       0.87       0.17            -0.48             0.19                 0.62                 1.21
had_qr_code_menu_scans_before_and_didnt_last_month  -0.35       0.71       0.33            -1.00             0.31                 0.37                 1.36
had_qr_code_menu_scans_last_month                   -0.36       0.70       0.22            -0.78             0.07                 0.46                 1.07
had_reservations_submissions_before_and_didnt_l...   0.17       1.19       0.27            -0.36             0.71                 0.69                 2.03
had_reservations_submissions_last_month             -0.49       0.61       0.33            -1.14           

# Model 2: changed_picture_or_text

...

<lifelines.CoxTimeVaryingFitter: fitted with 31703 periods, 2527 subjects, 211 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2527
 number of periods = 31703
  number of events = 211
partial log-likelihood = -1318.45
  time fit was run = 2021-12-21 11:26:37 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.04       0.96       0.02            -0.09            -0.00                 0.92                 1.00
Monthly                                              0.92       2.50       0.24             0.45             1.39                 1.56                 4.00
New.email.subscriber.signups.last.month.div10       -0.13       0.88       0.05            -0.23            -0.02                 0.79                 0.98
SpotHopper_site_not_live_yet                         0.38       1.47       0.26            -0.12             0.89                 0.88                 2.43
had_added_events_manually_edited_events_before_...  -0.00       1.00       0.19            -0.37             0.36                 0.69                 1.44
had_added_events_manually_edited_events_last_3_...  -0.61       0.54       0.29            -1.18            -0.04                 0.31                 0.96
had_catering_submissions_before_and_didnt_last_...   0.08       1.08       0.26            -0.43             0.58                 0.65                 1.79
had_catering_submissions_last_3_months              -0.59       0.56       0.23            -1.04            -0.13                 0.35                 0.88
had_clicked_emails_before_and_didnt_last_3_months    0.63       1.87       0.18             0.27             0.99                 1.31                 2.68
had_clicked_emails_last_3_months                     0.10       1.11       0.20            -0.29             0.49                 0.75                 1.63
had_emails_sent_manually_scheduled_emails_sent_...   0.07       1.07       0.17            -0.26             0.40                 0.77                 1.50
had_emails_sent_manually_scheduled_emails_sent_...  -0.29       0.75       0.18            -0.65             0.07                 0.52                 1.08
had_number_of_requests_for_new_text_fragment_be...   0.24       1.27       0.17            -0.09             0.57                 0.91                 1.78
had_number_of_requests_for_new_text_fragment_la...   0.54       1.71       0.29            -0.03             1.10                 0.97                 3.01
had_online_orders_before_and_didnt_last_month        0.50       1.65       0.31            -0.12             1.12                 0.89                 3.06
had_online_orders_last_month                        -0.21       0.81       0.33            -0.86             0.44                 0.42                 1.56
had_posts_liked_before_and_didnt_last_3_months       0.23       1.26       0.18            -0.12             0.58                 0.89                 1.78
had_posts_liked_last_3_months                       -0.40       0.67       0.24            -0.86             0.07                 0.42                 1.07
had_posts_on_facebook_before_and_didnt_last_month    0.33       1.39       0.18            -0.03             0.69                 0.97                 1.99
had_posts_on_facebook_last_month                    -0.16       0.85       0.18            -0.52             0.19                 0.60                 1.21
had_posts_seen_before_and_didnt_last_3_months        0.41       1.50       0.20             0.01             0.81                 1.01                 2.24
had_posts_seen_last_3_months                         0.07       1.07       0.21            -0.35           

# Model 33: New.email.subscribers.allinclusive.last.month.div10

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 31703 periods, 2527 subjects, 211 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 2527
 number of periods = 31703
  number of events = 211
partial log-likelihood = -1318.98
  time fit was run = 2021-12-21 11:27:21 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Fb.page.likes.analysis.avg.log2                     -0.05       0.95       0.02            -0.09            -0.00                 0.92                 1.00
Monthly                                              0.98       2.66       0.24             0.51             1.45                 1.66                 4.26
New.email.subscribers.allinclusive.last.month.d...  -0.05       0.95       0.03            -0.10             0.00                 0.90                 1.00
SpotHopper_site_not_live_yet                         0.36       1.44       0.25            -0.14             0.86                 0.87                 2.37
Website.Views.last.month.total.log2                 -0.06       0.94       0.05            -0.16             0.03                 0.86                 1.03
had_added_events_manually_edited_events_before_...   0.01       1.01       0.19            -0.36             0.37                 0.70                 1.45
had_added_events_manually_edited_events_last_3_...  -0.61       0.54       0.29            -1.18            -0.04                 0.31                 0.96
had_catering_submissions_before_and_didnt_last_...   0.08       1.08       0.26            -0.43             0.59                 0.65                 1.80
had_catering_submissions_last_3_months              -0.56       0.57       0.23            -1.02            -0.10                 0.36                 0.90
had_clicked_emails_before_and_didnt_last_3_months    0.63       1.88       0.18             0.27             0.99                 1.31                 2.70
had_clicked_emails_last_3_months                     0.09       1.10       0.20            -0.30             0.48                 0.74                 1.62
had_emails_sent_manually_scheduled_emails_sent_...   0.07       1.07       0.17            -0.27             0.40                 0.77                 1.49
had_emails_sent_manually_scheduled_emails_sent_...  -0.29       0.75       0.18            -0.65             0.07                 0.52                 1.07
had_number_of_requests_for_new_text_fragment_be...   0.25       1.29       0.17            -0.08             0.59                 0.92                 1.80
had_number_of_requests_for_new_text_fragment_la...   0.54       1.72       0.29            -0.03             1.11                 0.97                 3.02
had_online_orders_before_and_didnt_last_month        0.50       1.65       0.31            -0.12             1.12                 0.89                 3.05
had_online_orders_last_month                        -0.13       0.88       0.34            -0.79             0.53                 0.45                 1.70
had_posts_liked_before_and_didnt_last_3_months       0.23       1.26       0.18            -0.12             0.57                 0.89                 1.78
had_posts_liked_last_3_months                       -0.39       0.68       0.24            -0.85             0.08                 0.43                 1.08
had_posts_on_facebook_before_and_didnt_last_month    0.32       1.38       0.18            -0.03             0.68                 0.97                 1.97
had_posts_on_facebook_last_month                    -0.18       0.84       0.18            -0.53             0.18                 0.59                 1.19
had_posts_seen_before_and_didnt_last_3_months        0.41       1.50       0.20             0.01           